In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import xlrd

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [78]:
mnist.train.images.shape
#images_matrix=mnist.train.images.reshape([55000,28,28,1])
#labels=mnist.train.labels
size=28
d_Data=1
N=55000
c1=5
d1=64
c2=5
d2=32
f=1024
m=10
batch=256
n_batch=(int)(mnist.train.num_examples/batch)

I am using two convolutional layer with each associated to a pooling layer and relu layer, and finally plus a fully-connected layer.
The input is 28*28*1.
the cnn1:kernel c1*c1*1*d1 stride=1
28*28*d1
the pooling1: 2*2 stride 2:
14*14*d1
the cnn2:kernel c2*c2*d1*d2
14*14*d2
pooling2:2,2 stride:2
7*7*d2
fully-connected:
wf(7*7*d2,f)
softmax: w(f,10)

In [79]:
tf.reset_default_graph() 
g1=tf.Graph()

In [80]:
with g1.as_default():
    Data=tf.placeholder(tf.float32,[batch,size,size,1],name="Images")
    Labels=tf.placeholder(tf.float32,[batch,m],name="Labels")
    with tf.variable_scope('cnn1'):
        W=tf.get_variable(name="kernel",dtype=tf.float32,shape=[c1,c1,d_Data,d1])
        B=tf.get_variable(name="bias1",dtype=tf.float32,initializer=tf.random_normal([d1],0.01))
        Cov1=tf.nn.conv2d(Data,W,[1,1,1,1],'SAME')
        Cov_b1=Cov1+B
        relu1=tf.nn.relu(Cov_b1,name="relu")
        max_pooling1=tf.nn.max_pool(relu1,[1,2,2,1],[1,2,2,1],"VALID",name="pool1")
        

In [81]:
with g1.as_default():
    with tf.variable_scope('cnn2'):
        W2=tf.get_variable(name="kernel2",dtype=tf.float32,shape=[c2,c2,d1,d2])
        B2=tf.get_variable(name="bias2",dtype=tf.float32,initializer=tf.random_normal([d2],0.01))
        Cov2=tf.nn.conv2d(max_pooling1,W2,[1,1,1,1],'SAME')
        Cov_b2=Cov2+B2
        relu2=tf.nn.relu(Cov_b2,name="relu2")
        max_pooling2=tf.nn.max_pool(relu2,[1,2,2,1],[1,2,2,1],"VALID",name="pool2")
        
        

In [82]:
with g1.as_default():
    with tf.variable_scope('fc'):
        W3=tf.get_variable(name="weight_fc",dtype=tf.float32,shape=[7*7*d2,f])
        trans=tf.reshape(max_pooling2,[batch,7*7*d2],name="fc_conv_convert")
        logisfc=tf.matmul(trans,W3)
        relufc=tf.nn.relu(logisfc,name="relu_fc")
    with tf.variable_scope("predict_training"):
        W4=tf.get_variable(name="weight_pred",dtype=tf.float32,shape=[f,m])
        logis_pred=tf.matmul(relufc,W4)
        SoftMaxLoss_vector=tf.nn.softmax_cross_entropy_with_logits(logits=logis_pred,labels=Labels)
        SoftMaxLoss=tf.reduce_sum(SoftMaxLoss_vector)
        optimizer=tf.train.AdamOptimizer(0.1,0.9,0.999,1e-08).minimize(SoftMaxLoss)
        init=tf.global_variables_initializer()


        
    

In [83]:
sess=tf.Session(graph=g1)
sess.run(init)
#for i in range(1):
 #   sess.run(optimizer,{Data:images_matrix,Labels:labels})
for i in range(2):
    for j in range(n_batch):
        images_v,labels=mnist.train.next_batch(batch)
        images_matrix=images_v.reshape([batch,28,28,1])
        sess.run(optimizer,{Data:images_matrix,Labels:labels})

In [84]:
[w,b,w2,b2,w3,w4]=sess.run([W,B,W2,B2,W3,W4])

In [85]:
[w,b,w2,b2,w3,w4]

[array([[[[-0.52036089, -0.45939422, -0.52095902, ..., -0.49755624,
           -0.74631381, -0.66000658]],
 
         [[-0.54485458, -0.53271461, -0.49353606, ..., -0.53747672,
           -0.63747555, -0.69894505]],
 
         [[-0.52153629, -0.52880043, -0.52209896, ..., -0.72413838,
           -0.69984877, -0.70911723]],
 
         [[-0.70760953, -0.47788849, -0.51427531, ..., -0.67324549,
           -0.68936294, -0.65577817]],
 
         [[-0.77974427, -0.62173456, -0.50500697, ..., -0.70586503,
           -0.72376025, -0.61903131]]],
 
 
        [[[-0.51941502, -0.47303823, -0.49327275, ..., -0.50487906,
           -0.71834469, -0.44107521]],
 
         [[-0.53506494, -0.4556362 , -0.55538201, ..., -0.44655272,
           -0.62194216, -0.45491859]],
 
         [[-0.47468472, -0.52611136, -0.54719979, ..., -0.62788504,
           -0.6108948 , -0.6001482 ]],
 
         [[-0.7303468 , -0.52836823, -0.46564734, ..., -0.63585734,
           -0.67070985, -0.60918963]],
 
         [[-0.69

In [69]:
sess.close()

In [70]:
batch_te=5000

In [71]:

g2=tf.Graph()
with g2.as_default():
    Data=tf.placeholder(tf.float32,[batch_te,size,size,1],name="Images")
    Labels=tf.placeholder(tf.float32,[batch_te,m],name="Labels")
    with tf.variable_scope('cnn1'):
        W=tf.placeholder(tf.float32,[c1,c1,d_Data,d1],name="W1")
        B=tf.placeholder(tf.float32,[d1],name="Bias1")
        Cov1=tf.nn.conv2d(Data,W,[1,1,1,1],'SAME')
        Cov_b1=Cov1+B
        relu1=tf.nn.relu(Cov_b1,name="relu")
        max_pooling1=tf.nn.max_pool(relu1,[1,2,2,1],[1,2,2,1],"VALID",name="pool1")
    with tf.variable_scope('cnn2'):
        W2=tf.placeholder(tf.float32,[c2,c2,d1,d2],name="W2")
        B2=tf.placeholder(tf.float32,[d2],name="Bias2")
        Cov2=tf.nn.conv2d(max_pooling1,W2,[1,1,1,1],'SAME')
        Cov_b2=Cov2+B2
        relu2=tf.nn.relu(Cov_b2,name="relu2")
        max_pooling2=tf.nn.max_pool(relu2,[1,2,2,1],[1,2,2,1],"VALID",name="pool2")
    with tf.variable_scope('fc'):
        W3=tf.placeholder(tf.float32,[7*7*d2,f],name="W3")
        trans=tf.reshape(max_pooling2,[batch_te,7*7*d2],name="fc_conv_convert")
        logisfc=tf.matmul(trans,W3)
        relufc=tf.nn.relu(logisfc,name="relu_fc")
    with tf.variable_scope("predict"):
        W4=tf.placeholder(tf.float32,[f,m],name="W4")
        logis_pred=tf.matmul(relufc,W4)
        prob=tf.nn.softmax(logis_pred)
        pred=tf.argmax(prob,1)
        labelte=tf.argmax(Labels,1)
        compare=tf.equal(pred,labelte)
        accuracy=tf.reduce_mean(tf.cast(compare,tf.float32))
        init=tf.global_variables_initializer()


        

In [72]:
sess2=tf.Session(graph=g2)
sess2.run(init)
#images_te_v=mnist.test.images
images_te_v=mnist.validation.images
#labels_te=mnist.test.labels
labels_te=mnist.validation.labels
images_te_matrix=images_te_v.reshape([batch_te,28,28,1])
a=sess2.run(accuracy,{Data:images_te_matrix,Labels:labels_te,W:w,B:b,W2:w2,B2:b2,W3:w3,W4:w4})

In [73]:
a

0.2054

In [74]:
w

array([[[[ -3.76649573e-03,  -4.39668715e-01,   1.06576793e-02, ...,
           -4.61300611e-02,   4.08359058e-02,  -5.40240288e-01]],

        [[ -6.68447837e-03,  -5.30542433e-01,   2.96349637e-02, ...,
            3.10400687e-02,   6.64882362e-04,  -5.42334557e-01]],

        [[ -5.14139198e-02,  -6.44768476e-01,  -5.31000160e-02, ...,
            4.90850024e-02,  -4.40234318e-03,  -5.67820847e-01]],

        [[  4.05587368e-02,  -6.26420021e-01,  -2.64528207e-02, ...,
            3.89723666e-02,   7.91341439e-03,  -6.39043391e-01]],

        [[  4.88875695e-02,  -6.47570431e-01,  -4.76204641e-02, ...,
           -5.34785278e-02,   1.46362670e-02,  -6.35291934e-01]]],


       [[[  2.77627818e-02,  -5.06926000e-01,  -1.31057240e-02, ...,
           -6.24461472e-03,  -3.91423926e-02,  -6.36299133e-01]],

        [[  1.99630074e-02,  -4.65814918e-01,  -1.11301132e-02, ...,
            5.18638268e-03,   3.24934535e-02,  -6.29509389e-01]],

        [[ -2.81707160e-02,  -6.63296938e-01, 

In [75]:
w[:,:,0,0]

array([[-0.0037665 , -0.00668448, -0.05141392,  0.04055874,  0.04888757],
       [ 0.02776278,  0.01996301, -0.02817072, -0.01376723,  0.05213055],
       [ 0.00479498,  0.00059907,  0.0023749 ,  0.0187961 , -0.00566206],
       [-0.02076871, -0.04985677, -0.05332261, -0.04570087,  0.05122518],
       [-0.05590469,  0.04865866, -0.03738509, -0.05531341, -0.04091815]], dtype=float32)